# Fitness: (Median of overheads, Median of routing costs )

## Load data


In [3]:
%matplotlib inline 
%reload_ext autoreload
from notebooks_commons import get_raw_data


# The first time we want to download the data from an index (might take a while!), 
# we should put the parameter load_from_db to True. Then, we can set it to False to 
# read the saved data from the local pickle file.
def read_data(index, load_from_db = False):
    print "\nReading from index " + index
    pair = get_raw_data(index, load_from_db)
    rtx_runs = pair[0]
    data     = pair[1] 
    return rtx_runs, data


# Here we specify the name of the index to read data from

large_index = "thomas-nsga2-vm-large" 



###
load_from_db = False

### Get Novelty data
large_runs, large_data = read_data(large_index, load_from_db)



Reading from index thomas-nsga2-vm-large
data retrieved from file raw_data/thomas-nsga2-vm-large.pickle


## Load Fitness Function

In [4]:
use_median = True

'''
Returns the fitness of an individual
'''
def get_fitness(ind):
    if use_median:
        return ind["payload"]["median_overhead"], ind["payload"]["median_routing"]
    else: 
        return ind["avg_overhead"], ind["avg_routing"]
            
'''
Returns the overhead of an individual
'''
def get_overhead(ind):
    if use_median:
        return ind["payload"]["median_overhead"]
    else:
        return ind["avg_overhead"]
    

'''
Returns the routing of an individual
'''
def get_routing(ind):
    if use_median:
        return ind["payload"]["median_routing"]
    else:
        return ind["avg_routing"]
    



def set_fitness_med(ind, overhead, routing):
    ind["payload"]["median_overhead"] = overhead
    ind["payload"]["median_routing"] = routing      

        
def set_fitness_avg(ind, overhead, routing):
    ind["avg_overhead"] = overhead
    ind["avg_routing"] = routing        

    
print("Fitness Function Loaded")

Fitness Function Loaded


## Check what's in there

In [5]:
import pprint
from IPython.display import Markdown, display

def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))
    
pp = pprint.PrettyPrinter(indent=4)

def check_data(rtx_runs, data):
    # sort according to seed 
    rtx_runs.sort(key=lambda d : d["seed"])
    opt_method = "N/A"
    if len(rtx_runs) > 0:
        try:
            opt_method = rtx_runs[0]["strategy"]["optimizer_method"]
        except:
            # mlr does not store the opt method name in field strategy.optimizer_method
            opt_method = "BOGP"
    print "There were " + str(len(rtx_runs)) + " runs performed by " + opt_method

    for rtx_run in rtx_runs:
        data_for_run = [d for d in data if d["parent"] == rtx_run["id"]]
        data_for_run.sort(key=lambda d : (d["_source"]["iteration"], d["_source"]["individual"]))
        printmd(str(len(data_for_run)) + "\t\t| seed " + str(rtx_run["seed"]) 
                + " | id " + str(rtx_run["id"]), "red")

        #for d in data_for_run:
        #    s = d["_source"]
        #    overheads = s["payload"]["overheads"]
        #    routings = s["payload"]["routings"]
        #    printmd("Iteration " + str(s["iteration"]) + ", individual " 
        #            + str(s["individual"]) + " with configuration", "blue")        
        #    pp.pprint(s["knobs"])
        #    printmd("has " + str(len(overheads)) + " overheads and " 
        #            + str(len(routings)) + " routings", "green")



check_data(large_runs, large_data)



There were 2 runs performed by NSGAII


<span style='color:red'>721		| seed 1 | id AWg3yk2D-DH7UHKUO40M</span>

<span style='color:red'>1003		| seed 2 | id AWg3ymDV-DH7UHKUO40N</span>

## Code for computing the hypervolume

In [6]:
#    https://ls11-www.cs.tu-dortmund.de/rudolph/hypervolume/start

#    Copyright (C) 2010 Simon Wessing
#    TU Dortmund University
#
#    This program is free software: you can redistribute it and/or modify
#    it under the terms of the GNU General Public License as published by
#    the Free Software Foundation, either version 3 of the License, or
#    (at your option) any later version.
#
#    This program is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#    GNU General Public License for more details.
#
#    You should have received a copy of the GNU General Public License
#    along with this program.  If not, see <http://www.gnu.org/licenses/>.


__author__ = "Simon Wessing"


class HyperVolume:
    """
    Hypervolume computation based on variant 3 of the algorithm in the paper:
    C. M. Fonseca, L. Paquete, and M. Lopez-Ibanez. An improved dimension-sweep
    algorithm for the hypervolume indicator. In IEEE Congress on Evolutionary
    Computation, pages 1157-1163, Vancouver, Canada, July 2006.

    Minimization is implicitly assumed here!

    """

    def __init__(self, referencePoint):
        """Constructor."""
        self.referencePoint = referencePoint
        self.list = []


    def compute(self, front):
        """Returns the hypervolume that is dominated by a non-dominated front.

        Before the HV computation, front and reference point are translated, so
        that the reference point is [0, ..., 0].

        """

        def weaklyDominates(point, other):
            for i in xrange(len(point)):
                if point[i] > other[i]:
                    return False
            return True

        relevantPoints = []
        referencePoint = self.referencePoint
        dimensions = len(referencePoint)
        for point in front:
            # only consider points that dominate the reference point
            if weaklyDominates(point, referencePoint):
                relevantPoints.append(point)
        if any(referencePoint):
            # shift points so that referencePoint == [0, ..., 0]
            # this way the reference point doesn't have to be explicitly used
            # in the HV computation
            for j in xrange(len(relevantPoints)):
                relevantPoints[j] = [relevantPoints[j][i] - referencePoint[i] for i in xrange(dimensions)]
        self.preProcess(relevantPoints)
        bounds = [-1.0e308] * dimensions
        hyperVolume = self.hvRecursive(dimensions - 1, len(relevantPoints), bounds)
        return hyperVolume


    def hvRecursive(self, dimIndex, length, bounds):
        """Recursive call to hypervolume calculation.

        In contrast to the paper, the code assumes that the reference point
        is [0, ..., 0]. This allows the avoidance of a few operations.

        """
        hvol = 0.0
        sentinel = self.list.sentinel
        if length == 0:
            return hvol
        elif dimIndex == 0:
            # special case: only one dimension
            # why using hypervolume at all?
            return -sentinel.next[0].cargo[0]
        elif dimIndex == 1:
            # special case: two dimensions, end recursion
            q = sentinel.next[1]
            h = q.cargo[0]
            p = q.next[1]
            while p is not sentinel:
                pCargo = p.cargo
                hvol += h * (q.cargo[1] - pCargo[1])
                if pCargo[0] < h:
                    h = pCargo[0]
                q = p
                p = q.next[1]
            hvol += h * q.cargo[1]
            return hvol
        else:
            remove = self.list.remove
            reinsert = self.list.reinsert
            hvRecursive = self.hvRecursive
            p = sentinel
            q = p.prev[dimIndex]
            while q.cargo != None:
                if q.ignore < dimIndex:
                    q.ignore = 0
                q = q.prev[dimIndex]
            q = p.prev[dimIndex]
            while length > 1 and (q.cargo[dimIndex] > bounds[dimIndex] or q.prev[dimIndex].cargo[dimIndex] >= bounds[dimIndex]):
                p = q
                remove(p, dimIndex, bounds)
                q = p.prev[dimIndex]
                length -= 1
            qArea = q.area
            qCargo = q.cargo
            qPrevDimIndex = q.prev[dimIndex]
            if length > 1:
                hvol = qPrevDimIndex.volume[dimIndex] + qPrevDimIndex.area[dimIndex] * (qCargo[dimIndex] - qPrevDimIndex.cargo[dimIndex])
            else:
                qArea[0] = 1
                qArea[1:dimIndex+1] = [qArea[i] * -qCargo[i] for i in xrange(dimIndex)]
            q.volume[dimIndex] = hvol
            if q.ignore >= dimIndex:
                qArea[dimIndex] = qPrevDimIndex.area[dimIndex]
            else:
                qArea[dimIndex] = hvRecursive(dimIndex - 1, length, bounds)
                if qArea[dimIndex] <= qPrevDimIndex.area[dimIndex]:
                    q.ignore = dimIndex
            while p is not sentinel:
                pCargoDimIndex = p.cargo[dimIndex]
                hvol += q.area[dimIndex] * (pCargoDimIndex - q.cargo[dimIndex])
                bounds[dimIndex] = pCargoDimIndex
                reinsert(p, dimIndex, bounds)
                length += 1
                q = p
                p = p.next[dimIndex]
                q.volume[dimIndex] = hvol
                if q.ignore >= dimIndex:
                    q.area[dimIndex] = q.prev[dimIndex].area[dimIndex]
                else:
                    q.area[dimIndex] = hvRecursive(dimIndex - 1, length, bounds)
                    if q.area[dimIndex] <= q.prev[dimIndex].area[dimIndex]:
                        q.ignore = dimIndex
            hvol -= q.area[dimIndex] * q.cargo[dimIndex]
            return hvol


    def preProcess(self, front):
        """Sets up the list data structure needed for calculation."""
        dimensions = len(self.referencePoint)
        nodeList = MultiList(dimensions)
        nodes = [MultiList.Node(dimensions, point) for point in front]
        for i in xrange(dimensions):
            self.sortByDimension(nodes, i)
            nodeList.extend(nodes, i)
        self.list = nodeList


    def sortByDimension(self, nodes, i):
        """Sorts the list of nodes by the i-th value of the contained points."""
        # build a list of tuples of (point[i], node)
        decorated = [(node.cargo[i], node) for node in nodes]
        # sort by this value
        decorated.sort()
        # write back to original list
        nodes[:] = [node for (_, node) in decorated]
            
            
            
class MultiList: 
    """A special data structure needed by FonsecaHyperVolume. 
    
    It consists of several doubly linked lists that share common nodes. So, 
    every node has multiple predecessors and successors, one in every list.

    """

    class Node: 
        
        def __init__(self, numberLists, cargo=None): 
            self.cargo = cargo 
            self.next  = [None] * numberLists
            self.prev = [None] * numberLists
            self.ignore = 0
            self.area = [0.0] * numberLists
            self.volume = [0.0] * numberLists
    
        def __str__(self): 
            return str(self.cargo)
        
        
    def __init__(self, numberLists):  
        """Constructor. 
        
        Builds 'numberLists' doubly linked lists.

        """
        self.numberLists = numberLists
        self.sentinel = MultiList.Node(numberLists)
        self.sentinel.next = [self.sentinel] * numberLists
        self.sentinel.prev = [self.sentinel] * numberLists  
        
        
    def __str__(self):
        strings = []
        for i in xrange(self.numberLists):
            currentList = []
            node = self.sentinel.next[i]
            while node != self.sentinel:
                currentList.append(str(node))
                node = node.next[i]
            strings.append(str(currentList))
        stringRepr = ""
        for string in strings:
            stringRepr += string + "\n"
        return stringRepr
    
    
    def __len__(self):
        """Returns the number of lists that are included in this MultiList."""
        return self.numberLists
    
    
    def getLength(self, i):
        """Returns the length of the i-th list."""
        length = 0
        sentinel = self.sentinel
        node = sentinel.next[i]
        while node != sentinel:
            length += 1
            node = node.next[i]
        return length
            
            
    def append(self, node, index):
        """Appends a node to the end of the list at the given index."""
        lastButOne = self.sentinel.prev[index]
        node.next[index] = self.sentinel
        node.prev[index] = lastButOne
        # set the last element as the new one
        self.sentinel.prev[index] = node
        lastButOne.next[index] = node
        
        
    def extend(self, nodes, index):
        """Extends the list at the given index with the nodes."""
        sentinel = self.sentinel
        for node in nodes:
            lastButOne = sentinel.prev[index]
            node.next[index] = sentinel
            node.prev[index] = lastButOne
            # set the last element as the new one
            sentinel.prev[index] = node
            lastButOne.next[index] = node
        
        
    def remove(self, node, index, bounds): 
        """Removes and returns 'node' from all lists in [0, 'index'[."""
        for i in xrange(index): 
            predecessor = node.prev[i]
            successor = node.next[i]
            predecessor.next[i] = successor
            successor.prev[i] = predecessor  
            if bounds[i] > node.cargo[i]:
                bounds[i] = node.cargo[i]
        return node
    
    
    def reinsert(self, node, index, bounds):
        """
        Inserts 'node' at the position it had in all lists in [0, 'index'[
        before it was removed. This method assumes that the next and previous 
        nodes of the node that is reinserted are in the list.

        """
        for i in xrange(index): 
            node.prev[i].next[i] = node
            node.next[i].prev[i] = node
            if bounds[i] > node.cargo[i]:
                bounds[i] = node.cargo[i]         
     
    

print("Hypervolume module loaded.")

Hypervolume module loaded.


## Compute fitness and store it

In [15]:
import sys
import numpy as np
import pylab 
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc("savefig", dpi=300)
# plt.rcParams["figure.figsize"] = [8,6]


## params 
iterations_default = 100
iterations_mlr = iterations_default * 10
sample_size = 5000
total_number_of_docs = 1000
debug = False

## maintain iterations to consider
iterations = iterations_default
def set_iterations(opt_method):
    global iterations
    if opt_method == "BOGP":
        iterations = iterations_mlr
    else:
        iterations = iterations_default 
        
        
'''
Prints an individuals
'''
def print_individual(ind):
    print("Iteration: " + str(ind["iteration"]) 
                  + " | Individual: " + str(ind["individual"]) 
                  + " | Overhead: " + str(get_overhead(ind)) 
                  + " | Routing: " + str(get_routing(ind)))
    print("\tConfiguration:")
    for knob, knob_value in ind["knobs"].iteritems():
        print("\t\t" + str(knob) + ": " + str(knob_value))

    
def get_config_key(ind):
    knobs = ind["knobs"]
    key = str(knobs["re_route_every_ticks"]) + ","
    key = key + str(knobs["freshness_cut_off_value"]) + ","
    key = key + str(knobs["average_edge_duration_factor"]) + ","
    key = key + str(knobs["exploration_percentage"]) + ","
    key = key + str(knobs["freshness_update_factor"]) + ","
    key = key + str(knobs["route_random_sigma"]) + ","
    key = key + str(knobs["max_speed_and_length_factor"])
    return key
  
    
'''
Calculate the rolling/moving average
'''
def calculate_moving_average(aList):
    moving_avg = 0
    tmp_count = 0
    for r in range(len(aList)):
        moving_avg = (moving_avg * tmp_count + aList[r]) / (tmp_count + 1)
        tmp_count = tmp_count + 1
    return moving_avg


'''
Computes the fitness of all individuals of the given runs and 
stores the fitness of an individual to the individual.
Returns a dictionary where the key is the id of the run and the
value are all individuals of this run.
'''
def compute_fitness(rtx_runs, data):
    global worst_overhead
    global worst_routing
    global best_overhead
    global best_routing
        
    run_to_all_individuals = dict()
    # counter how often evaluations have been reused over all rtx runs
    number_of_reuse_per_run = []
    
    # for each run
    for rtx_run in rtx_runs:
        rtx_run_id = rtx_run["id"]
        # get all documents = all evaluations
        all_documents = [d["_source"] for d in data if d["parent"] == rtx_run_id]
        try:
            opt_method = rtx_run["strategy"]["optimizer_method"]
        except:
            # mlr does not store the opt method name in field strategy.optimizer_method
            opt_method = "BOGP"
        # set iterations
        set_iterations(opt_method)
        
        if debug:
            print("================================================================")
        print("Run: " + rtx_run_id + " | " + opt_method 
              + " | seed: " + str(rtx_run["seed"]) 
              + " | number of docs/evals: " + str(len(all_documents))
              + " | iterations to process: " + str(iterations))
        if debug:
            print("================================================================")

            
        # counter how often evaluations have been reused for this rtx run
        number_of_reuse = 0
        # all inidividuals across all iterations
        all_individuals = []
        # stored fitnesses for reuse calculated with the fitness function using avg
        fitness_store = dict()
        
        # for each iteration
        for i in range(0, iterations):
            # get documents of the given iteration for the given run
            documents_of_iteration = [d for d in all_documents if d["iteration"] == i]
            pop_size = len(documents_of_iteration)
            if debug:
                print("Iteration: " + str(i) + " | Population size: " + str(pop_size))

            # for each individual of the iteration
            for j in range(len(documents_of_iteration)):        
                next_individuals = [d for d in documents_of_iteration if d["individual"] == j]
                if len(next_individuals) > 1:
                    print("ERROR: more than one individual with same number within one iteration.")
                else:
                    individual = next_individuals[0] # list of one elem

                # get overheads of individual
                overheads = individual["payload"]["overheads"]
                # get routings of individual
                routings = individual["payload"]["routings"]
                if overheads == [-1]:
                    # reused fitness evaluations are stored in these fields although 
                    # the fitness are not the average but the medians. 
                    med_overhead = individual["payload"]["avg_overhead"]  
                    med_routing = individual["payload"]["avg_routing"]
                    # store properly "median" fitness values in the individual
                    set_fitness_med(individual, med_overhead, med_routing)
                    # get fitness for average overhead and routing
                    individual_key = get_config_key(individual)
                    avg_overhead, avg_routing = fitness_store[individual_key]
                    set_fitness_avg(individual, avg_overhead, avg_routing) 
                    if debug:
                        print("Individual: " + individual_key + " | load fitness: " 
                              + str(avg_overhead) + " " + str(avg_routing))
                    # stats
                    number_of_reuse = number_of_reuse + 1
                else:
                    # sanity checks -- fitness needs not to be computed and stored; it is already there
                    if len(overheads) <> sample_size:
                        print("ERROR: Overhead entries " + str(len(overheads)) 
                              + "<> sample size " + str(sample_size))
                    med_overhead = np.median(overheads) 
                    med_routing = np.median(routings)
                    individual["payload"]["median_overhead"] = med_overhead
                    individual["payload"]["median_routing"] = med_routing
                    #med_overhead = individual["payload"]["median_overhead"]
                    #med_routing = individual["payload"]["median_routing"]
                    #if med_overhead <> computed_med_overhead:
                    #    print("Overhead does not match: " 
                    #          + str(med_overhead) + " vs " + str(computed_med_overhead))
                    #if med_routing <> computed_med_routing:
                    #    print("Routing does not match: " 
                    #          + str(med_routing) + " vs " + str(computed_med_routing))
                        
                    # compute and store average values
                    avg_overhead = np.mean(overheads)
                    avg_routing = np.mean(routings) # calculate_moving_average(routings)
                    set_fitness_avg(individual, avg_overhead, avg_routing)
                    individual_key = get_config_key(individual)
                    fitness_store[individual_key] = avg_overhead, avg_routing
                    if debug:
                        print("Individual: " + individual_key + " | store fitness: " 
                              + str(avg_overhead) + " " + str(avg_routing))
               
                # use med or avg from now on
                if use_median:   
                    overhead = med_overhead
                    routing = med_routing
                else:
                    overhead = avg_overhead
                    routing = avg_routing
                
                # check for worst and best cases
                if overhead > worst_overhead:
                    worst_overhead = overhead
                if overhead < best_overhead:
                    best_overhead = overhead
                if routing > worst_routing:
                    worst_routing = routing
                if routing < best_routing:
                    best_routing = routing
                    

                if debug:
                    print("Individual: " + str(individual["individual"]) 
                         + " | Overhead: " + str(get_overhead(individual))
                         + " | Routing: " + str(get_routing(individual)))

                all_individuals.append(individual)
                # next individual

            # next iteration
            if debug:
                print("")

        #if len(all_documents) == total_number_of_docs:
        run_to_all_individuals[rtx_run_id] = all_individuals
        #else:
         #   print("Skip run for further analysis, only " + str(len(all_documents)) 
          #        + " documents/evaluations present.")
        number_of_reuse_per_run.append(number_of_reuse)
        # next run
    if len(number_of_reuse_per_run) > 0:
        print("Min | Average | Max number of reused evaluations per run: "
              + str(np.min(number_of_reuse_per_run)) + " | "
              + str(np.mean(number_of_reuse_per_run)) + " | "
              + str(np.max(number_of_reuse_per_run)) + " | "
              + "\n==========================================================")
    return run_to_all_individuals
      

# worst and best objective values achieved in *all* runs
worst_overhead = sys.float_info.min
worst_routing = sys.float_info.min
best_overhead = sys.float_info.max
best_routing = sys.float_info.max


# check / store fitness
large_run_inds = compute_fitness(large_runs, large_data)


Run: AWg3yk2D-DH7UHKUO40M | NSGAII | seed: 1 | number of docs/evals: 721 | iterations to process: 100
Run: AWg3ymDV-DH7UHKUO40N | NSGAII | seed: 2 | number of docs/evals: 1003 | iterations to process: 100
Min | Average | Max number of reused evaluations per run: 220 | 367.0 | 514 | 


## Analyze and plot data

In [16]:
import sys
import numpy as np
import pylab 
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc("savefig", dpi=300)
# plt.rcParams["figure.figsize"] = [8,6]


'''
Method to take a list of individuals and return just the individuals which lie 
on the Pareto frontier, sorted into order.
Default behaviour is to find the maximum for both X and Y objectives, but the option is
available to specify maxX = False or maxY = False to find the minimum for either
or both of the objectves.
Adapted from: http://oco-carbon.com/metrics/find-pareto-frontiers-in-python/
'''
def get_pareto_front(Inds, maxX = False, maxY = False):
    # Sort the list in either ascending or descending order of X
    Inds.sort(key=lambda x: get_overhead(x), reverse=maxX)
    # Start the Pareto frontier with the first value in the sorted list
    p_front = [Inds[0]]    
    # Loop through the sorted list
    for ind in Inds[1:]:
        if maxY: 
            # changed >= to >
            if get_routing(ind) > get_routing(p_front[-1]): # Look for higher values of Y… 
                p_front.append(ind) # … and add them to the Pareto frontier
        else:
            # changed <= to <
            if get_routing(ind) < get_routing(p_front[-1]): # Look for lower values of Y…    
                p_front.append(ind) # … and add them to the Pareto frontier
    return p_front


def analyze_and_plot(rtx_runs, run_to_all_individuals, plotting = False):    
    printmd("===========================================================", "red")
    # pareto fronts of all runs
    pareto_fronts_of_all_runs = []
    
    opt_method = ""
    if len(rtx_runs) > 0:
        try:
            opt_method = rtx_runs[0]["strategy"]["optimizer_method"]
        except:
            # mlr does not store the opt method name in field strategy.optimizer_method
            opt_method = "BOGP"

    # set iterations
    set_iterations(opt_method)
    
    # for each run
    for rtx_run in rtx_runs:
        rtx_run_id = rtx_run["id"]
        # get all individuals of the run
        all_individuals = run_to_all_individuals.get(rtx_run_id, None)
        if all_individuals is None:
            print("Encountered skipped run " + str(rtx_run_id))
        else:
            print("\nRun: " + rtx_run_id + " | method: " + opt_method 
                  + " | seed: " + str(rtx_run["seed"]) 
                  + " | number of documents/evaluations: " + str(len(all_individuals)))

            # compute pareto front of the run ####################################################
            pareto_front_of_run = get_pareto_front(all_individuals)
            pareto_fronts_of_all_runs.append(pareto_front_of_run)
            if plotting:
                print("\nPareto front of the run:")
                for p in pareto_front_of_run:
                    print_individual(p)

            # compute hypervolume ####################################################
            reference_point = [worst_overhead, worst_routing]
            print("Reference Point: " + str(reference_point))
            hyperVolume = HyperVolume(reference_point)

            pareto_front_values = [[get_overhead(el), get_routing(el)] for el in pareto_front_of_run]
            hv = hyperVolume.compute(pareto_front_values)
            print("Hypervolume: " + str(hv))

            # plotting #################################################################
            if plotting:

                # Hypervolume over fitness evaluations ###########################
                fig, axes = plt.subplots()
                fig.suptitle('Evolution of the Hypervolume', fontsize=16)    
                plt.xlabel('Fitness Evaluations') 
                x = range(total_number_of_docs)
                plt.ylabel('Hypervolume')
                y = []

                current_pareto_front = []
                for i in range(iterations):
                    new_inds = [el for el in all_individuals if el["iteration"] == i]
                    if not isinstance(new_inds, (list,)):
                        new_inds = [new_inds]
                    for j in range(len(new_inds)):
                        # new_ind is a list with one element
                        new_ind = [el for el in new_inds if el["individual"] == j]
                        if len(new_ind) > 1:
                            print("ERROR: More than one individual with the same number within the same iteration.")
                        # debug_msg = debug_msg 
                        #   + "(" + str(new_ind[0]["iteration"]) + ", " + str(new_ind[0]["individual"]) + ")"
                        current_pareto_front.append(new_ind[0])
                        current_pareto_front = get_pareto_front(current_pareto_front)
                        current_pareto_front_values = [[get_overhead(el), get_routing(el)] 
                                                       for el in current_pareto_front]
                        hv = hyperVolume.compute(current_pareto_front_values)
                        y.append(hv)

                plt.plot(x,y, color='black', label='Hypervolume')
                pylab.legend(loc='best')

                # Overhead over fitness evaluations ###############################          
                fig, axes = plt.subplots()
                fig.suptitle('Evolution of Overhead', fontsize=16)    
                plt.xlabel('Fitness Evaluations') 
                x = range(total_number_of_docs)
                plt.ylabel('Overhead')
                y = []

                min_overhead_over_iterations = []
                best_min = sys.float_info.max
                for i in range(iterations):
                    new_inds = [el for el in all_individuals if el["iteration"] == i]
                    if not isinstance(new_inds, (list,)):
                        new_inds = [new_inds]
                    for j in range(len(new_inds)):
                        # new_ind is a list with one element
                        new_ind = [el for el in new_inds if el["individual"] == j]
                        if len(new_ind) > 1:
                            print("ERROR: More than one individual with the same number within the same iteration.")    

                        overhead = get_overhead(new_ind[0])
                        y.append(overhead)
                        if overhead < best_min:
                            best_min = overhead
                        min_overhead_over_iterations.append(best_min)

                plt.scatter(x,y, marker="+", color='black', label='individual')
                plt.scatter(x, min_overhead_over_iterations, s=100, facecolors='none', 
                            edgecolors='r', label='perato front for overhead')
                pylab.legend(loc='best')

                # Routing over fitness evaluations ########################################
                fig, axes = plt.subplots()
                fig.suptitle('Evolution of Routing', fontsize=16)    
                plt.xlabel('Fitness Evaluations') 
                x = range(total_number_of_docs)
                plt.ylabel('Routing')
                y = []

                min_routing_over_iterations = []
                best_min = sys.float_info.max
                for i in range(iterations):
                    new_inds = [el for el in all_individuals if el["iteration"] == i]
                    if not isinstance(new_inds, (list,)):
                        new_inds = [new_inds]
                    for j in range(len(new_inds)):
                        # new_ind is a list with one element
                        new_ind = [el for el in new_inds if el["individual"] == j]
                        if len(new_ind) > 1:
                            print("ERROR: More than one individual with the same number within the same iteration.")    

                        routing = get_routing(new_ind[0])
                        y.append(routing)
                        if routing < best_min:
                            best_min = routing
                        min_routing_over_iterations.append(best_min)

                plt.scatter(x,y, marker="+", color='black', label='individual')
                plt.scatter(x, min_routing_over_iterations, s=100, 
                            facecolors='none', edgecolors='r', label='perato front for routing')
                pylab.legend(loc='best')


                # print all individuals and pareto front ##############################
                fig, axes = plt.subplots()
                # axes.grid(True)
                fig.suptitle('All individuals and the pareto front', fontsize=16)
                overheads = [get_overhead(el) for el in all_individuals]
                routings = [get_routing(el) for el in all_individuals] 

                plt.ylabel('Routing')
                plt.xlabel('Overhead')
                plt.scatter(overheads, routings, marker="+", color='black', label='Individual')
                p_front_overheads = [get_overhead(el) for el in pareto_front_of_run]
                p_front_routings = [get_routing(el) for el in pareto_front_of_run] 
                if len(pareto_front_of_run) > 1:
                    plt.plot(p_front_overheads, p_front_routings, label="Pareto Front")
                else:
                    plt.scatter(p_front_avg_o, p_front_avg_p, label="Pareto Front")
                pylab.legend(loc='best')
                #for i,j in zip(avg_o,avg_p):
                #    axes.annotate(str(i)+", "+str(j),xy=(i,j))

                plt.show()

    return pareto_fronts_of_all_runs


# analyze and plot data

large_pfronts = analyze_and_plot(large_runs, large_run_inds)


<span style='color:red'>===========================================================</span>


Run: AWg3yk2D-DH7UHKUO40M | method: NSGAII | seed: 1 | number of documents/evaluations: 721
Reference Point: [1.8451166012704037, 154.0]
Hypervolume: 32.5651395558

Run: AWg3ymDV-DH7UHKUO40N | method: NSGAII | seed: 2 | number of documents/evaluations: 1003
Reference Point: [1.8451166012704037, 154.0]
Hypervolume: 33.4227298665


## Analysis of the hypervolume and objectives over multiple runs

In [19]:
from scipy import stats

reference_point = [2.2170968676925815, 311.0] # [worst_overhead, worst_routing]
hyperVolume = HyperVolume(reference_point)

'''
Computes the hypervolumes for all pareto fronts
'''
def compute_hvs(pfronts):
    hvs = []
    for pfront in pfronts:
        pfront_values = [[get_overhead(el), get_routing(el)] for el in pfront]
        hv = hyperVolume.compute(pfront_values)
        hvs.append(hv)       
    return hvs


'''
Computes the average hypervolume for all pareto fronts
'''
def compute_avg_hv(pfronts):
    hvs = compute_hvs(pfronts)      
    print "Hypervolume " + str(hvs)
    return np.mean(hvs)


'''
Computes the median hypervolume for all pareto fronts
'''
def compute_median_hv(pfronts, plotting=False):
    hvs = compute_hvs(pfronts)       
    print "Hypervolume " + str(hvs)
    return np.median(hvs)


'''
Computes the average of the overhead and routing over all pareto fronts of the all runs
'''
def compute_avg_objectives(pfronts):
    overheads, routings = get_objectives_for_each_pfront(pfronts)
    print "Overheads / Routings " + str(overheads) + " / " + str(routings)
    return np.mean(overheads), np.mean(routings)

'''
Computes the median of the overhead and routing over all pareto fronts of the all runs
'''
def compute_median_objectives(pfronts):
    overheads, routings = get_objectives_for_each_pfront(pfronts)
    print "Overheads / Routings " + str(overheads) + " / " + str(routings)
    return np.median(overheads), np.median(routings)


def get_objectives_for_each_pfront(pfronts, plotting=False):
    overheads = []
    routings = []
    # for each run/seed
    
    for pfront in pfronts:
        overheads_single_run = [get_overhead(el) for el in pfront]
        overheads.append(np.min(overheads_single_run))
        routings_single_run = [get_routing(el) for el in pfront]
        routings.append(np.min(routings_single_run))
    
    return overheads, routings


'''
List of lists, one for each run containing how the hypervolume evolves with each fitness evaluation.
'''
def compute_hv_over_evals(rtx_runs, run_to_all_individuals, opt_method):
    # set iterations
    set_iterations(opt_method)
    
    hv_series_of_all_runs = []
    for rtx_run in rtx_runs:
        rtx_run_id = rtx_run["id"]
        # get all individuals of the run
        all_individuals = run_to_all_individuals.get(rtx_run_id, None)
        if all_individuals is None:
            # print("No documents for run " + str(rtx_run_id))
            break
       
        # debug_msg = ""
        hv_series = []          
        current_pareto_front = []
        for i in range(iterations):
            new_inds = [el for el in all_individuals if el["iteration"] == i]
            if not isinstance(new_inds, (list,)):
                new_inds = [new_inds]
            for j in range(len(new_inds)):
                # new_ind is a list with one element
                new_ind = [el for el in new_inds if el["individual"] == j]
                if len(new_ind) > 1:
                    print("ERROR: More than one individual with the same number within the same iteration.")
                # debug_msg = debug_msg 
                #   + "(" + str(new_ind[0]["iteration"]) + ", " + str(new_ind[0]["individual"]) + ")"
                current_pareto_front.append(new_ind[0])
                current_pareto_front = get_pareto_front(current_pareto_front)
                current_pareto_front_values = [[get_overhead(el), get_routing(el)] 
                                               for el in current_pareto_front]
                hv = hyperVolume.compute(current_pareto_front_values)
                hv_series.append(hv)
        # print(debug_msg)
        hv_series_of_all_runs.append(hv_series)
    return hv_series_of_all_runs
    

'''
Plots the evolution of the hypervolume of the methods over fitness evaluations. 
The hypervolume plotted is the mean hypervolume over the runs for each method.
'''
def plot_hypervolume_evolution(random_rtx_runs, random_run_to_all_individuals,
                               nsga2_rtx_runs, nsga2_run_to_all_individuals,
                               novelty_rtx_runs, novelty_run_to_all_individuals,
                               mlr_rtx_runs, mlr_run_to_all_individuals, cars_number):
    methods = ["BOGP", "NSGA-II", "Novelty Search", "Random"]
    linestyles = [':', '-', '-.', '--']
    
    mlr_hv_series = compute_hv_over_evals(mlr_rtx_runs, mlr_run_to_all_individuals, methods[0])
    nsga2_hv_series = compute_hv_over_evals(nsga2_rtx_runs, nsga2_run_to_all_individuals, methods[1])
    novelty_hv_series = compute_hv_over_evals(novelty_rtx_runs, novelty_run_to_all_individuals, methods[2])
    random_hv_series = compute_hv_over_evals(random_rtx_runs, random_run_to_all_individuals, methods[3])
    
    all_hv_series = [mlr_hv_series, nsga2_hv_series, novelty_hv_series, random_hv_series]
    
    fig, axes = plt.subplots()
    # fig.suptitle("Evolution of the Mean Hypervolume (" + str(cars_number) + " cars)", fontsize=16)    
    plt.ylabel('Mean Hypervolume')
    plt.xlabel('Fitness Evaluations')
    
    for i in range(len(methods)):
        method = methods[i]
        hv_series = all_hv_series[i]
        a = np.array(hv_series)   
        plot_data = np.mean(a, axis=0)
        print(method + " - final mean hv: " + str(plot_data[len(plot_data)-1]))
        plt.plot(range(len(plot_data)), plot_data, color='black', linestyle=linestyles[i], label=method)
     
    pylab.legend(loc='best')
    plt.show()

    
def plot_hypervolume_boxplots(random_pfronts, nsga2_pfronts, novelty_pfronts, mlr_pfronts, cars_number):
    hvs_random = compute_hvs(random_pfronts)
    hvs_nsga2 = compute_hvs(nsga2_pfronts)
    hvs_novelty = compute_hvs(novelty_pfronts)
    hvs_mlr = compute_hvs(mlr_pfronts)

    hvs = [hvs_mlr, hvs_nsga2, hvs_novelty, hvs_random]
    hvs_names = ["BOGP", "NSGA-II", "Novelty S.", "Random"]
    hvs_labels = range(1,5)

    fig,ax = plt.subplots()
    # plt.title(str(cars_number) + " cars")
    ax.boxplot(hvs, 0, '', positions=hvs_labels)
    for i in range(len(hvs)):
        ax.plot(hvs_labels[i], np.mean(hvs[i]), ".", label='mean', color='black', linestyle=':')
    plt.xticks(hvs_labels, hvs_names) 
    plt.ylabel('Hypervolume')
    plt.show() 
    
    # statistical tests
    run_ttest(hvs, hvs_names)
    
    # distributions
    for i in range(len(hvs)):
        fig,ax = plt.subplots()
        plt.hist(hvs[i], bins=30)  
        plt.title(str(hvs_names[i]) + " (" + str(cars_number) + " cars)")
        plt.ylabel('frequency')
        plt.xlabel('hypervolume')
        plt.show() 

    
def plot_objectives_boxplots(random_pfronts, nsga2_pfronts, novelty_pfronts, mlr_pfronts, cars_number):
    overheads_random, routings_random = get_objectives_for_each_pfront(random_pfronts) 
    overheads_nsga2, routings_nsga2 = get_objectives_for_each_pfront(nsga2_pfronts) 
    overheads_novelty, routings_novelty = get_objectives_for_each_pfront(novelty_pfronts)  
    overheads_mlr, routings_mlr = get_objectives_for_each_pfront(mlr_pfronts)
        
    overheads = [overheads_mlr, overheads_nsga2, overheads_novelty, overheads_random]
    routings  = [routings_mlr, routings_nsga2, routings_novelty, routings_random]
    names = ["BOGP", "NSGA-II", "Novelty S.", "Random"]
    labels = range(1,5)
    
    # Trip overhead
    fig,ax = plt.subplots()
    # plt.title(str(cars_number) + " cars")
    ax.boxplot(overheads, 0, '', positions=labels)
    for i in range(len(overheads)):
        ax.plot(labels[i], np.mean(overheads[i]), ".", label='mean', color='black', linestyle=':')
    plt.xticks(labels, names) 
    plt.ylabel('Trip Overhead')
    plt.show()
    
    # statistical test #############
    print("Trip Overhead:")
    run_ttest(overheads, names)
    
    # distributions
    for i in range(len(overheads)):
        fig,ax = plt.subplots()
        plt.hist(overheads[i], bins=30)  
        plt.title("Distribution: " + str(names[i]) + " (" + str(cars_number) + " cars)")
        plt.ylabel('frequency')
        plt.xlabel('trip overhead')
    plt.show()    
    
    # Routing Cost
    fig,ax = plt.subplots()
    # plt.title(str(cars_number) + " cars")
    ax.boxplot(routings, 0, '', positions=labels)
    for i in range(len(routings)):
        ax.plot(labels[i], np.mean(routings[i]), ".", label='mean', color='black', linestyle=':')
    plt.xticks(labels, names) 
    plt.ylabel('Routing Cost')
    plt.show() 
        
    # statistical test #############
    print("Routing Cost:")
    run_ttest(routings, names)
    
     # distributions
    for i in range(len(routings)):
        fig,ax = plt.subplots()
        plt.hist(routings[i], bins=30)  
        plt.title("Distribution: " + str(names[i]) + " (" + str(cars_number) + " cars)")
        plt.ylabel('frequency')
        plt.xlabel('routing')
    plt.show() 
    

def run_ttest(dataXXX, names):
    alpha = 0.05
    for i in range(len(dataXXX)):
        data_first = dataXXX[i]
        name_first = names[i]
        for j in range(len(names)):
            if j == i:
                break
            data_second = dataXXX[j]
            name_second = names[j]
            statistic, pvalue = stats.ttest_ind(data_first, data_second, equal_var = False)
            statistic2, pvalue2 = stats.wilcoxon(data_first, data_second)
            
            different_averages = bool(pvalue <= alpha)
            is_is_not = "\tis\t" if different_averages else "\tis not\t"
            print(name_first + is_is_not 
                  + " statistically significantly different than "+ name_second + " (ttest)")
            
            different_averages2 = bool(pvalue2 <= alpha)
            is_is_not2 = "\tis\t" if different_averages2 else "\tis not\t"
            print(name_first + is_is_not2 
                  + " statistically significantly different than "+ name_second + " (wilcoxon)")


            
def check_avg(pfronts, name):
    avg_hv = compute_avg_hv(pfronts)
    median_hv = compute_median_hv(pfronts)
    
    avg_best_objectives = compute_avg_objectives(pfronts)
    avg_best_overhead = avg_best_objectives[0]
    avg_best_routing = avg_best_objectives[1]
    
    med_best_objectives = compute_median_objectives(pfronts)
    med_best_overhead = med_best_objectives[0]
    med_best_routing = med_best_objectives[1]
    
    tt = "\t\t" if name == "BOGP" else "\t"
        
        
    print(name + " ("+str(len(pfronts))+" runs):" + tt
          + str(avg_best_overhead) + "\t| " + str(med_best_overhead) + "\t| " 
          + str(avg_best_routing) + "\t| " + str(med_best_routing) + "\t| " 
          + str(avg_hv) + "\t|" + str(median_hv) + "\t| ")
    

    
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

print("Iterations considered: " + str(iterations))
print("Average Overhead (Best of each run) | Median Overhead (Best of each run)" 
      +" | Average Routing (Best of each run) | Median Routing (Best of each run)" 
      +" |Average Hypervolume | Median Hypervolume" )

check_avg(large_pfronts, "NSGA-II")



Iterations considered: 100
Average Overhead (Best of each run) | Median Overhead (Best of each run) | Average Routing (Best of each run) | Median Routing (Best of each run) |Average Hypervolume | Median Hypervolume
Hypervolume [183.76384325222881, 186.14059115144792]
Hypervolume [183.76384325222881, 186.14059115144792]
Overheads / Routings [1.6165508617445932, 1.6068746987602869] / [1.0, 1.0]
Overheads / Routings [1.6165508617445932, 1.6068746987602869] / [1.0, 1.0]
NSGA-II (2 runs):	1.61171278025	| 1.61171278025	| 1.0	| 1.0	| 184.952217202	|184.952217202	| 


### 